# In this approach I will try to extract  Part of Speech using spacy lib

# Assumption: In tweet text, ther's a lot of noise and the true sentiment of the whole statement might be distinctly resprented  by  some part of speech : Adj, Noun, Propernoun

With that assumption, I will extract POS and use them to predict sentiment

limititaion: To extract POS, use a lot of utility so I need to cut down the training set which might affect the accuracy rate. 
: (

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import nltk
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import re
import string
from sklearn.model_selection import train_test_split

In [2]:
import sys
!{sys.executable} -m pip install spacy
!{sys.executable} -m spacy download en
!{sys.executable} pip install more-itertools

2021-12-23 21:21:28.080405: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'cudart64_110.dll'; dlerror: cudart64_110.dll not found
2021-12-23 21:21:28.080455: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


[!] As of spaCy v3.0, shortcuts like 'en' are deprecated. Please use the full
pipeline package name 'en_core_web_sm' instead.
[+] Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


C:\Python\python.exe: can't open file 'E:\DM_HW2Kaggle\pip': [Errno 2] No such file or directory


In [3]:
!pip install more-itertools

In [12]:
from nltk.corpus import wordnet

In [13]:
import itertools 

In [14]:
emo_df = pd.read_csv("emotion.csv")

In [4]:
import spacy
nlp = spacy.load("en_core_web_sm")
doc = nlp("National Tsing Hua University was selected as one of the seven national research universities in Taiwan")
pos = [(token.text,token.pos_)for token in doc]
print(pos)

[('National', 'PROPN'), ('Tsing', 'PROPN'), ('Hua', 'PROPN'), ('University', 'PROPN'), ('was', 'AUX'), ('selected', 'VERB'), ('as', 'ADP'), ('one', 'NUM'), ('of', 'ADP'), ('the', 'DET'), ('seven', 'NUM'), ('national', 'ADJ'), ('research', 'NOUN'), ('universities', 'NOUN'), ('in', 'ADP'), ('Taiwan', 'PROPN')]


In [16]:
tweets_DM= pd.read_pickle("tweets_DM_clean")

In [17]:
tweets_DM

,_score,_index,_source,_crawldate,_type,tweet.hashtags,tweet.tweet_id,tweet.text,clean_text
0,391,hashtag_tweets,"{'tweet': {'hashtags': ['Snapchat'], 'tweet_id...",2015-05-23 11:42:47,tweets,[Snapchat],0x376b20,"People who post ""add me on #Snapchat"" must be ...",people who post add me on snapchat must be deh...
1,433,hashtag_tweets,"{'tweet': {'hashtags': ['freepress', 'TrumpLeg...",2016-01-28 04:52:09,tweets,"[freepress, TrumpLegacy, CNN]",0x2d5350,"@brianklaas As we see, Trump is dangerous to #...",as we see trump is dangerous to freepress aro...
2,232,hashtag_tweets,"{'tweet': {'hashtags': ['bibleverse'], 'tweet_...",2017-12-25 04:39:20,tweets,[bibleverse],0x28b412,"Confident of your obedience, I write to you, k...",confident of your obedience i write to you kno...
3,376,hashtag_tweets,"{'tweet': {'hashtags': [], 'tweet_id': '0x1cd5...",2016-01-24 23:53:05,tweets,[],0x1cd5b0,Now ISSA is stalking Tasha 😂😂😂 <LH>,now issa is stalking tasha 😂😂
4,989,hashtag_tweets,"{'tweet': {'hashtags': [], 'tweet_id': '0x2de2...",2016-01-08 17:18:59,tweets,[],0x2de201,"""Trust is not the same as faith. A friend is s...",trust is not the same as faith a friend is som...
...,...,...,...,...,...,...,...,...,...
1867530,827,hashtag_tweets,"{'tweet': {'hashtags': ['mixedfeeling', 'butim...",2015-05-12 12:51:52,tweets,"[mixedfeeling, butimTHATperson]",0x316b80,When you buy the last 2 tickets remaining for ...,when you buy the last tickets remaining for a...
1867531,368,hashtag_tweets,"{'tweet': {'hashtags': [], 'tweet_id': '0x29d0...",2017-10-02 17:54:04,tweets,[],0x29d0cb,I swear all this hard work gone pay off one da...,i swear all this hard work gone pay off one da...
1867532,498,hashtag_tweets,"{'tweet': {'hashtags': [], 'tweet_id': '0x2a6a...",2016-10-10 11:04:32,tweets,[],0x2a6a4f,@Parcel2Go no card left when I wasn't in so I ...,no card left when i wasn't in so i have no id...
1867533,840,hashtag_tweets,"{'tweet': {'hashtags': [], 'tweet_id': '0x24fa...",2016-09-02 14:25:06,tweets,[],0x24faed,"Ah, corporate life, where you can date <LH> us...",ah corporate life where you can date using ju...


In [18]:
# Becasue I cannot extract POS from a big data set, I cut down them by the shuffling. 
#In addition, a number of 'emotion:joy' is greater than the other significantly and the anger is the least 
#prepare TRAIN_dataset for the shuffle
tweets_DM_train = tweets_DM.merge(emo_df, how='inner',left_on = 'tweet.tweet_id', right_on = 'tweet_id')

In [17]:
tweets_DM_train

,_score,_index,_source,_crawldate,_type,tweet.hashtags,tweet.tweet_id,tweet.text,clean_text,tweet_id,emotion
0,391,hashtag_tweets,"{'tweet': {'hashtags': ['Snapchat'], 'tweet_id...",2015-05-23 11:42:47,tweets,[Snapchat],0x376b20,"People who post ""add me on #Snapchat"" must be ...",people who post add me on snapchat must be deh...,0x376b20,anticipation
1,433,hashtag_tweets,"{'tweet': {'hashtags': ['freepress', 'TrumpLeg...",2016-01-28 04:52:09,tweets,"[freepress, TrumpLegacy, CNN]",0x2d5350,"@brianklaas As we see, Trump is dangerous to #...",as we see trump is dangerous to freepress aro...,0x2d5350,sadness
2,376,hashtag_tweets,"{'tweet': {'hashtags': [], 'tweet_id': '0x1cd5...",2016-01-24 23:53:05,tweets,[],0x1cd5b0,Now ISSA is stalking Tasha 😂😂😂 <LH>,now issa is stalking tasha 😂😂,0x1cd5b0,fear
3,120,hashtag_tweets,"{'tweet': {'hashtags': ['authentic', 'LaughOut...",2015-06-11 04:44:05,tweets,"[authentic, LaughOutLoud]",0x1d755c,@RISKshow @TheKevinAllison Thx for the BEST TI...,thx for the best time tonight what stories h...,0x1d755c,joy
4,1021,hashtag_tweets,"{'tweet': {'hashtags': [], 'tweet_id': '0x2c91...",2015-08-18 02:30:07,tweets,[],0x2c91a8,Still waiting on those supplies Liscus. <LH>,still waiting on those supplies liscus,0x2c91a8,anticipation
...,...,...,...,...,...,...,...,...,...,...,...
1455558,94,hashtag_tweets,"{'tweet': {'hashtags': ['NoWonder', 'Happy'], ...",2016-12-26 02:44:07,tweets,"[NoWonder, Happy]",0x321566,I'm SO HAPPY!!! #NoWonder the name of this sho...,i'm so happy nowonder the name of this show ha...,0x321566,joy
1455559,627,hashtag_tweets,"{'tweet': {'hashtags': [], 'tweet_id': '0x3895...",2015-04-01 08:14:56,tweets,[],0x38959e,In every circumtance I'd like to be thankful t...,in every circumtance i'd like to be thankful t...,0x38959e,joy
1455560,274,hashtag_tweets,"{'tweet': {'hashtags': ['blessyou'], 'tweet_id...",2016-11-17 23:46:22,tweets,[blessyou],0x2cbca6,there's currently two girls walking around the...,there's currently two girls walking around the...,0x2cbca6,joy
1455561,840,hashtag_tweets,"{'tweet': {'hashtags': [], 'tweet_id': '0x24fa...",2016-09-02 14:25:06,tweets,[],0x24faed,"Ah, corporate life, where you can date <LH> us...",ah corporate life where you can date using ju...,0x24faed,joy


In [18]:
tweets_DMsf_train = tweets_DM_train[['_score','tweet_id','clean_text','tweet.hashtags','emotion']]

In [19]:
#do the shuffle
unshuffled_inputs = tweets_DMsf_train.iloc[:,:4].to_numpy()
unshuffled_outputs = tweets_DMsf_train.iloc[:,-1].to_numpy()

In [20]:
tweets_DMsf_train.shape[0] , unshuffled_inputs.shape[0]

(1455563, 1455563)

In [21]:
first_shuffle_indices = np.arange(unshuffled_inputs.shape[0])
np.random.shuffle(first_shuffle_indices)

shuffled_inputs = unshuffled_inputs[first_shuffle_indices]
shuffled_outputs = unshuffled_outputs[first_shuffle_indices]
shuffled_outputs

array(['joy', 'anticipation', 'sadness', ..., 'joy', 'anticipation',
       'disgust'], dtype=object)

In [22]:
lowest_number = 39712  #this corresponds to the lowest number of items for any given emotion (this case anger)

number_anger, number_anticipation, number_sadness, number_fear = 0, 0, 0, 0
number_joy, number_trust, number_disgust, number_surprise = 0, 0, 0, 0
indices_to_remove = []

for i in range(shuffled_outputs.shape[0]):
    #anticipation:
    if(shuffled_outputs[i]=='anticipation'):
        number_anticipation+= 1
        if(number_anticipation>lowest_number):
            indices_to_remove.append(i)
    #Sadness:
    if(shuffled_outputs[i]=='sadness'):
        number_sadness+= 1
        if(number_sadness>lowest_number):
            indices_to_remove.append(i)
    #Fear
    if(shuffled_outputs[i]=='fear'):
        number_fear+= 1
        if(number_fear>lowest_number):
            indices_to_remove.append(i)
    #Joy
    if(shuffled_outputs[i]=='joy'):
        number_joy+= 1
        if(number_joy>lowest_number):
            indices_to_remove.append(i)
    #Anger
    if(shuffled_outputs[i]=='anger'):
        number_anger+= 1
        if(number_anger>lowest_number):
            indices_to_remove.append(i)
    #Trust
    if(shuffled_outputs[i]=='trust'):
        number_trust+= 1
        if(number_trust>lowest_number):
            indices_to_remove.append(i)
    #Disgust:
    if(shuffled_outputs[i]=='disgust'):
        number_disgust+= 1
        if(number_disgust>lowest_number):
            indices_to_remove.append(i)
    #Surprise:
    if(shuffled_outputs[i]=='surprise'):
        number_surprise+= 1
        if(number_surprise>lowest_number):
            indices_to_remove.append(i)

In [23]:
scaled_inputs = np.delete(shuffled_inputs,indices_to_remove,axis=0)
scaled_outputs = np.delete(shuffled_outputs, indices_to_remove, axis = 0)

In [24]:
len(scaled_inputs), len(scaled_outputs)

(317696, 317696)

In [25]:
#Reshuffle the data:
shuffle_indices = np.arange(scaled_inputs.shape[0])
np.random.shuffle(shuffle_indices)

re_shuffled_inputs = scaled_inputs[shuffle_indices]
re_shuffled_outputs = scaled_outputs[shuffle_indices]

In [26]:
len(re_shuffled_inputs), len(re_shuffled_outputs)

(317696, 317696)

In [27]:
inputs_df = pd.DataFrame(re_shuffled_inputs)
outputs_df =  pd.DataFrame(re_shuffled_outputs)
final_df = pd.concat([inputs_df, outputs_df], axis=1)
final_df.columns = ['_score', 'tweet_id', 'text_clean','tweet.hashtags', 'emotion']
final_df

#shuffle completed

,_score,tweet_id,text_clean,tweet.hashtags,emotion
0,763,0x30aba4,abuse is evil the permits it,[ABUSE],disgust
1,429,0x1f79c6,it is written the prayer of a righteous person...,[],anticipation
2,33,0x2c6d6d,yes we have the time i didn't type that,[],surprise
3,797,0x326846,if the horse tells you that you're that's th...,[],surprise
4,954,0x213028,feed that ego sad,[Sad],sadness
...,...,...,...,...,...
317691,139,0x347c16,starting school soon so scared for my first ye...,"[freshman, haveanyadvice]",fear
317692,310,0x319ba1,today i am grateful for the positive relations...,[healthyCorporateCulture],trust
317693,338,0x1ebea2,days until i'm a year older and days to go u...,[BookRelease],anticipation
317694,868,0x2634cb,corey taylor or s loop arch enemy or radiohead...,[Theloop],anger


In [28]:
tweets_DMsf_train = final_df

In [208]:
# I add id column because there will be  a link back to this dataset after extraction of POS later
tweets_DMsf_train['id'] = tweets_DMsf_train.index + 0
tweets_DMsf_train

,_score,tweet_id,text_clean,tweet.hashtags,emotion,POS,POS2,id
0,763,0x30aba4,abuse is evil the permits it,[ABUSE],disgust,"(abuse, is, evil, , the, , permits, it)","(abuse, is, evil, , the, , permits, it)",0
1,429,0x1f79c6,it is written the prayer of a righteous person...,[],anticipation,"(it, is, written, the, prayer, of, a, righteou...","(it, is, written, the, prayer, of, a, righteou...",1
2,33,0x2c6d6d,yes we have the time i didn't type that,[],surprise,"( , yes, we, have, the, time, i, did, n't, ty...","( , yes, we, have, the, time, i, did, n't, ty...",2
3,797,0x326846,if the horse tells you that you're that's th...,[],surprise,"(if, the, horse, tells, you, that, you, 're, ...","(if, the, horse, tells, you, that, you, 're, ...",3
4,954,0x213028,feed that ego sad,[Sad],sadness,"( , feed, that, ego, , sad)","( , feed, that, ego, , sad)",4
...,...,...,...,...,...,...,...,...
317691,139,0x347c16,starting school soon so scared for my first ye...,"[freshman, haveanyadvice]",fear,"(starting, school, soon, so, scared, for, my, ...","(starting, school, soon, so, scared, for, my, ...",317691
317692,310,0x319ba1,today i am grateful for the positive relations...,[healthyCorporateCulture],trust,"(today, i, am, grateful, for, the, positive, r...","(today, i, am, grateful, for, the, positive, r...",317692
317693,338,0x1ebea2,days until i'm a year older and days to go u...,[BookRelease],anticipation,"( , days, until, i, 'm, a, year, older, and, ...","( , days, until, i, 'm, a, year, older, and, ...",317693
317694,868,0x2634cb,corey taylor or s loop arch enemy or radiohead...,[Theloop],anger,"(corey, taylor, or, s, loop, arch, enemy, or, ...","(corey, taylor, or, s, loop, arch, enemy, or, ...",317694


In [ ]:
import spacy
nlp = spacy.load("en_core_web_sm")
tweets_DMsf_train['POS']=tweets_DMsf_train['text_clean'].apply(lambda x: nlp(x))

nlp = spacy.load("en_core_web_sm")
doc = nlp(tweets_DMsf_train['POS'][0])
pos = [(token.text,token.pos_)for token in doc]
print(pos)

#spacy extract the noun chunk
doc = nlp(tweets_DMsf_train['POS'][0])
spacy.displacy.render(doc, style="ent")
doc_nouns = list(doc.noun_chunks)
print(doc_nouns)

In [172]:
docs = list(nlp.pipe(tweets_DMsf_train['text_clean']))

In [200]:
type(docs)

list

def extract_tokens_plus_meta(doc:spacy.tokens.doc.Doc):
    """Extract tokens and metadata from individual spaCy doc."""
    return [
        (i.text, i.i, i.lemma_, i.ent_type_, i.tag_, 
         i.dep_, i.pos_, i.is_stop, i.is_alpha, 
         i.is_digit, i.is_punct) for i in doc
    ]

In [202]:
#This function is used to implement POS-tokenization to pandas dataframe
def tidy_tokens(docs):
    """Extract tokens and metadata from list of spaCy docs."""
    
    cols = [
        "doc_id", "token", "token_order", "lemma", 
        "ent_type", "tag", "dep", "pos", "is_stop", 
        "is_alpha", "is_digit", "is_punct"
    ]
    
    meta_df = []
    for ix, doc in enumerate(docs):
        meta = extract_tokens_plus_meta(doc)
        meta = pd.DataFrame(meta)
        meta.columns = cols[1:]
        meta = meta.assign(doc_id = ix).loc[:, cols]
        meta_df.append(meta)
        
    return pd.concat(meta_df)   

In [205]:
a = tidy_tokens(docs)

In [209]:
pos = a

In [215]:
# GOT THAT POS FINALLY!!!
# connet back to tweets_DMsf_train by id
# filter out by pos first
pos[20:40]

,doc_id,token,token_order,lemma,ent_type,tag,dep,pos,is_stop,is_alpha,is_digit,is_punct
12,1,effective,12,effective,,JJ,conj,ADJ,False,True,False,False
13,1,this,13,this,,DT,det,DET,True,True,False,False
14,1,righteousness,14,righteousness,,NN,nsubjpass,NOUN,False,True,False,False
15,1,is,15,be,,VBZ,auxpass,AUX,True,True,False,False
16,1,given,16,give,,VBN,conj,VERB,False,True,False,False
17,1,through,17,through,,IN,prep,ADP,True,True,False,False
18,1,faith,18,faith,,NN,pobj,NOUN,False,True,False,False
19,1,in,19,in,,IN,prep,ADP,True,True,False,False
20,1,,20,,,_SP,dep,SPACE,False,False,False,False
21,1,christ,21,christ,,NNP,appos,PROPN,False,True,False,False


In [228]:
#filter only NOUN, ADJ, PROPN
a=pos[(pos['pos']== 'NOUN') | (pos['pos']== 'ADJ') | (pos['pos']== 'PROPN') ] 

In [230]:
pos_clean = a

In [232]:
pos_clean = a[['doc_id', 'lemma', 'pos']]

In [233]:
# connect back to tweets_DMsf_train by doc id
pos_clean

,doc_id,lemma,pos
0,0,abuse,NOUN
2,0,evil,ADJ
4,1,prayer,NOUN
7,1,righteous,ADJ
8,1,person,NOUN
...,...,...,...
9,317694,theloop,NOUN
0,317695,cecil,NOUN
7,317695,brightside,NOUN
9,317695,delusional,ADJ


In [234]:
# merge the POS BACK to main dataframe
tweets_DMsf_train_pos = tweets_DMsf_train.merge(pos_clean, how='inner',left_on = 'id', right_on = 'doc_id')

In [242]:
tweets_DMsf_train_pos

,_score,tweet_id,text_clean,tweet.hashtags,emotion,POS,POS2,id,doc_id,lemma,pos
0,763,0x30aba4,abuse is evil the permits it,[ABUSE],disgust,"(abuse, is, evil, , the, , permits, it)","(abuse, is, evil, , the, , permits, it)",0,0,abuse,NOUN
1,763,0x30aba4,abuse is evil the permits it,[ABUSE],disgust,"(abuse, is, evil, , the, , permits, it)","(abuse, is, evil, , the, , permits, it)",0,0,evil,ADJ
2,429,0x1f79c6,it is written the prayer of a righteous person...,[],anticipation,"(it, is, written, the, prayer, of, a, righteou...","(it, is, written, the, prayer, of, a, righteou...",1,1,prayer,NOUN
3,429,0x1f79c6,it is written the prayer of a righteous person...,[],anticipation,"(it, is, written, the, prayer, of, a, righteou...","(it, is, written, the, prayer, of, a, righteou...",1,1,righteous,ADJ
4,429,0x1f79c6,it is written the prayer of a righteous person...,[],anticipation,"(it, is, written, the, prayer, of, a, righteou...","(it, is, written, the, prayer, of, a, righteou...",1,1,person,NOUN
...,...,...,...,...,...,...,...,...,...,...,...
1519552,868,0x2634cb,corey taylor or s loop arch enemy or radiohead...,[Theloop],anger,"(corey, taylor, or, s, loop, arch, enemy, or, ...","(corey, taylor, or, s, loop, arch, enemy, or, ...",317694,317694,theloop,NOUN
1519553,723,0x242e56,cecil allowed only run that inning brightside...,"[brightside, delusional, stlcards]",joy,"(cecil, allowed, only, , run, that, inning, b...","(cecil, allowed, only, , run, that, inning, b...",317695,317695,cecil,NOUN
1519554,723,0x242e56,cecil allowed only run that inning brightside...,"[brightside, delusional, stlcards]",joy,"(cecil, allowed, only, , run, that, inning, b...","(cecil, allowed, only, , run, that, inning, b...",317695,317695,brightside,NOUN
1519555,723,0x242e56,cecil allowed only run that inning brightside...,"[brightside, delusional, stlcards]",joy,"(cecil, allowed, only, , run, that, inning, b...","(cecil, allowed, only, , run, that, inning, b...",317695,317695,delusional,ADJ


In [243]:
a = tweets_DMsf_train_pos[['tweet_id','text_clean','emotion','doc_id','lemma','pos']]

In [257]:
tweets_DMsf_train_pos

,_score,tweet_id,text_clean,tweet.hashtags,emotion,POS,POS2,id,doc_id,lemma,pos
0,763,0x30aba4,abuse is evil the permits it,[ABUSE],disgust,"(abuse, is, evil, , the, , permits, it)","(abuse, is, evil, , the, , permits, it)",0,0,abuse,NOUN
1,763,0x30aba4,abuse is evil the permits it,[ABUSE],disgust,"(abuse, is, evil, , the, , permits, it)","(abuse, is, evil, , the, , permits, it)",0,0,evil,ADJ
2,429,0x1f79c6,it is written the prayer of a righteous person...,[],anticipation,"(it, is, written, the, prayer, of, a, righteou...","(it, is, written, the, prayer, of, a, righteou...",1,1,prayer,NOUN
3,429,0x1f79c6,it is written the prayer of a righteous person...,[],anticipation,"(it, is, written, the, prayer, of, a, righteou...","(it, is, written, the, prayer, of, a, righteou...",1,1,righteous,ADJ
4,429,0x1f79c6,it is written the prayer of a righteous person...,[],anticipation,"(it, is, written, the, prayer, of, a, righteou...","(it, is, written, the, prayer, of, a, righteou...",1,1,person,NOUN
...,...,...,...,...,...,...,...,...,...,...,...
1519552,868,0x2634cb,corey taylor or s loop arch enemy or radiohead...,[Theloop],anger,"(corey, taylor, or, s, loop, arch, enemy, or, ...","(corey, taylor, or, s, loop, arch, enemy, or, ...",317694,317694,theloop,NOUN
1519553,723,0x242e56,cecil allowed only run that inning brightside...,"[brightside, delusional, stlcards]",joy,"(cecil, allowed, only, , run, that, inning, b...","(cecil, allowed, only, , run, that, inning, b...",317695,317695,cecil,NOUN
1519554,723,0x242e56,cecil allowed only run that inning brightside...,"[brightside, delusional, stlcards]",joy,"(cecil, allowed, only, , run, that, inning, b...","(cecil, allowed, only, , run, that, inning, b...",317695,317695,brightside,NOUN
1519555,723,0x242e56,cecil allowed only run that inning brightside...,"[brightside, delusional, stlcards]",joy,"(cecil, allowed, only, , run, that, inning, b...","(cecil, allowed, only, , run, that, inning, b...",317695,317695,delusional,ADJ


In [270]:
# link POS back to main df and groupby each doc_id and make them a list
a=a.groupby('doc_id')['lemma'].apply(list)

In [275]:
a

doc_id
0                                             [abuse, evil]
1         [prayer, righteous, person, powerful, effectiv...
2                                                    [time]
3                                              [horse, way]
4                                          [feed, ego, sad]
                                ...                        
317691    [school, scared, first, year, high, school, fr...
317692    [today, grateful, positive, relationship, boss...
317693    [day, year, old, day, release, september, hell...
317694       [corey, taylor, s, loop, arch, enemy, theloop]
317695             [cecil, brightside, delusional, stlcard]
Name: lemma, Length: 307751, dtype: object

In [260]:
 tweets_DMsf_train

,_score,tweet_id,text_clean,tweet.hashtags,emotion,POS,POS2,id
0,763,0x30aba4,abuse is evil the permits it,[ABUSE],disgust,"(abuse, is, evil, , the, , permits, it)","(abuse, is, evil, , the, , permits, it)",0
1,429,0x1f79c6,it is written the prayer of a righteous person...,[],anticipation,"(it, is, written, the, prayer, of, a, righteou...","(it, is, written, the, prayer, of, a, righteou...",1
2,33,0x2c6d6d,yes we have the time i didn't type that,[],surprise,"( , yes, we, have, the, time, i, did, n't, ty...","( , yes, we, have, the, time, i, did, n't, ty...",2
3,797,0x326846,if the horse tells you that you're that's th...,[],surprise,"(if, the, horse, tells, you, that, you, 're, ...","(if, the, horse, tells, you, that, you, 're, ...",3
4,954,0x213028,feed that ego sad,[Sad],sadness,"( , feed, that, ego, , sad)","( , feed, that, ego, , sad)",4
...,...,...,...,...,...,...,...,...
317691,139,0x347c16,starting school soon so scared for my first ye...,"[freshman, haveanyadvice]",fear,"(starting, school, soon, so, scared, for, my, ...","(starting, school, soon, so, scared, for, my, ...",317691
317692,310,0x319ba1,today i am grateful for the positive relations...,[healthyCorporateCulture],trust,"(today, i, am, grateful, for, the, positive, r...","(today, i, am, grateful, for, the, positive, r...",317692
317693,338,0x1ebea2,days until i'm a year older and days to go u...,[BookRelease],anticipation,"( , days, until, i, 'm, a, year, older, and, ...","( , days, until, i, 'm, a, year, older, and, ...",317693
317694,868,0x2634cb,corey taylor or s loop arch enemy or radiohead...,[Theloop],anger,"(corey, taylor, or, s, loop, arch, enemy, or, ...","(corey, taylor, or, s, loop, arch, enemy, or, ...",317694


In [276]:
b = tweets_DMsf_train.merge(a, how='right',left_on = 'id', right_on = 'doc_id')

In [19]:
b['lemma']

NameError: name 'b' is not defined

In [279]:
# use to convert list obj. to string
def listToString(s): 
    
    # initialize an empty string
    str1 = " " 
    
    # return string  
    return (str1.join(s))

In [280]:
b['POS3']=b['lemma'].apply(lambda x: listToString(x))

In [282]:
c = b[['tweet_id', 'text_clean', 'emotion','id','lemma','POS3']]

In [283]:
df_tweet_POS = c

In [285]:
df_tweet_POS.to_pickle("df_tweet_POS_complete.pkl")

In [286]:
df_tweet_POS['POS3']

,tweet_id,text_clean,emotion,id,lemma,POS3
0,0x30aba4,abuse is evil the permits it,disgust,0,"[abuse, evil]",abuse evil
1,0x1f79c6,it is written the prayer of a righteous person...,anticipation,1,"[prayer, righteous, person, powerful, effectiv...",prayer righteous person powerful effective rig...
2,0x2c6d6d,yes we have the time i didn't type that,surprise,2,[time],time
3,0x326846,if the horse tells you that you're that's th...,surprise,3,"[horse, way]",horse way
4,0x213028,feed that ego sad,sadness,4,"[feed, ego, sad]",feed ego sad
...,...,...,...,...,...,...
307746,0x347c16,starting school soon so scared for my first ye...,fear,317691,"[school, scared, first, year, high, school, fr...",school scared first year high school freshman ...
307747,0x319ba1,today i am grateful for the positive relations...,trust,317692,"[today, grateful, positive, relationship, boss...",today grateful positive relationship boss team...
307748,0x1ebea2,days until i'm a year older and days to go u...,anticipation,317693,"[day, year, old, day, release, september, hell...",day year old day release september hell month ...
307749,0x2634cb,corey taylor or s loop arch enemy or radiohead...,anger,317694,"[corey, taylor, s, loop, arch, enemy, theloop]",corey taylor s loop arch enemy theloop


In [289]:
df_iden = pd.read_csv("data_identification.csv")
df_iden

,tweet_id,identification
0,0x28cc61,test
1,0x29e452,train
2,0x2b3819,train
3,0x2db41f,test
4,0x2a2acc,train
...,...,...
1867530,0x227e25,train
1867531,0x293813,train
1867532,0x1e1a7e,train
1867533,0x2156a5,train


In [290]:
df_test= df_iden[df_iden['identification'] == 'test']
df_test=df_test.dropna()
df_test

,tweet_id,identification
0,0x28cc61,test
3,0x2db41f,test
15,0x2466f6,test
23,0x23f9e9,test
31,0x1fb4e1,test
...,...,...
1867495,0x2c4dc2,test
1867496,0x31be7c,test
1867500,0x1ca58e,test
1867515,0x35c8ba,test


In [291]:
tweets_DM_test =  tweets_DM.merge(df_test,how='inner',left_on = 'tweet.tweet_id', right_on = 'tweet_id')

In [292]:
tweets_DM_test['clean_text']

,_score,_index,_source,_crawldate,_type,tweet.hashtags,tweet.tweet_id,tweet.text,clean_text,tweet_id,identification
0,232,hashtag_tweets,"{'tweet': {'hashtags': ['bibleverse'], 'tweet_...",2017-12-25 04:39:20,tweets,[bibleverse],0x28b412,"Confident of your obedience, I write to you, k...",confident of your obedience i write to you kno...,0x28b412,test
1,989,hashtag_tweets,"{'tweet': {'hashtags': [], 'tweet_id': '0x2de2...",2016-01-08 17:18:59,tweets,[],0x2de201,"""Trust is not the same as faith. A friend is s...",trust is not the same as faith a friend is som...,0x2de201,test
2,66,hashtag_tweets,"{'tweet': {'hashtags': ['materialism', 'money'...",2015-09-09 09:22:55,tweets,"[materialism, money, possessions]",0x218443,When do you have enough ? When are you satisfi...,when do you have enough when are you satisfie...,0x218443,test
3,104,hashtag_tweets,"{'tweet': {'hashtags': ['GodsPlan', 'GodsWork'...",2015-10-10 14:33:26,tweets,"[GodsPlan, GodsWork]",0x2939d5,"God woke you up, now chase the day #GodsPlan #...",god woke you up now chase the day godsplan god...,0x2939d5,test
4,310,hashtag_tweets,"{'tweet': {'hashtags': [], 'tweet_id': '0x2628...",2016-10-23 08:49:50,tweets,[],0x26289a,"In these tough times, who do YOU turn to as yo...",in these tough times who do you turn to as you...,0x26289a,test
...,...,...,...,...,...,...,...,...,...,...,...
411967,602,hashtag_tweets,"{'tweet': {'hashtags': [], 'tweet_id': '0x2913...",2016-12-10 18:01:00,tweets,[],0x2913b4,"""For this is the message that ye heard from th...",for this is the message that ye heard from the...,0x2913b4,test
411968,598,hashtag_tweets,"{'tweet': {'hashtags': [], 'tweet_id': '0x2a98...",2015-01-04 14:40:55,tweets,[],0x2a980e,"""There is a lad here, which hath five barley l...",there is a lad here which hath five barley loa...,0x2a980e,test
411969,827,hashtag_tweets,"{'tweet': {'hashtags': ['mixedfeeling', 'butim...",2015-05-12 12:51:52,tweets,"[mixedfeeling, butimTHATperson]",0x316b80,When you buy the last 2 tickets remaining for ...,when you buy the last tickets remaining for a...,0x316b80,test
411970,368,hashtag_tweets,"{'tweet': {'hashtags': [], 'tweet_id': '0x29d0...",2017-10-02 17:54:04,tweets,[],0x29d0cb,I swear all this hard work gone pay off one da...,i swear all this hard work gone pay off one da...,0x29d0cb,test


In [293]:
# train dataset -tokenizing
#tokenizer - - using bag of words BOW 
from sklearn.feature_extraction.text import CountVectorizer

# build analyzers (bag-of-words)
BOW= CountVectorizer(tokenizer=nltk.word_tokenize) 

# apply analyzer to training data
BOW.fit(df_tweet_POS['POS3'])

train_data_BOW_train = BOW.transform(df_tweet_POS['POS3']) # USE  POS as a Train set
train_data_BOW_test = BOW.transform(tweets_DM_test['clean_text'])


C:\Python\lib\site-packages\sklearn\feature_extraction\text.py:516: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
C:\Python\lib\site-packages\sklearn\feature_extraction\text.py:1208: UserWarning: Upper case characters found in vocabulary while 'lowercase' is True. These entries will not be matched with any documents
  warnings.warn(


In [294]:
train_data_BOW_train.shape

(307751, 146866)

In [299]:
train_data_BOW_test.shape

(411972, 146866)

In [300]:
y_train = df_tweet_POS['emotion']

In [303]:
y_train

0              disgust
1         anticipation
2             surprise
3             surprise
4              sadness
              ...     
307746            fear
307747           trust
307748    anticipation
307749           anger
307750             joy
Name: emotion, Length: 307751, dtype: object

In [301]:
from sklearn.naive_bayes import MultinomialNB, GaussianNB, BernoulliNB
naivebay = MultinomialNB()
naive=naivebay.fit(train_data_BOW_train, y_train)

In [304]:
result = naive.predict(train_data_BOW_test)

In [306]:
result.shape

(411972,)

In [307]:
df_test['emotion'] = result

In [308]:
df_test.to

,tweet_id,identification,emotion
0,0x28cc61,test,anticipation
3,0x2db41f,test,anticipation
15,0x2466f6,test,trust
23,0x23f9e9,test,trust
31,0x1fb4e1,test,trust
...,...,...,...
1867495,0x2c4dc2,test,anticipation
1867496,0x31be7c,test,anticipation
1867500,0x1ca58e,test,sadness
1867515,0x35c8ba,test,anger


In [309]:
df_test[['tweet_id','emotion']].to_csv('POS')

In [ ]:
# resurlt = 14%  T_T  the training set might be too small. I guess